# PFAM Annotation Demo

Este notebook demuestra la funcionalidad completa del módulo de anotación PFAM de pyMut.

## Funcionalidades demostradas:
1. **Construcción de base de datos embebida** - Crear DuckDB con datos de PFAM y mappings
2. **Anotación de variantes** - Usar VariantAnnotator con backends pyvep/vep_cli
3. **Anotación de dominios PFAM** - Mapear variantes a dominios funcionales
4. **Resumen de dominios** - Análisis tipo maftools con pfam_domains()
5. **Ejemplos con VCF y MAF** - Casos de uso reales

## Requisitos
- Archivos de recursos en `src/pyMut/data/resources/`
- Datos de ejemplo: `tcga_laml.maf.gz` y `subset_50k_variants.vcf`


In [2]:
# Imports necesarios
import sys
import os
import pandas as pd
import numpy as np
from pathlib import Path

# Añadir src al path para importar pyMut
sys.path.insert(0, '../src')

# Importar funciones de PFAM annotation
from pyMut.pfam_annotation import (
    build_embedded_db,
    connect_db,
    VariantAnnotator,
    annotate_pfam,
    pfam_domains,
    get_resources_path,
    get_db_path
)

# Configurar pandas para mejor visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

print("✅ Imports completados")
print(f"📁 Ruta de recursos: {get_resources_path()}")
print(f"🗄️ Ruta de base de datos: {get_db_path()}")


✅ Imports completados
📁 Ruta de recursos: /home/luisruimore/Escritorio/pyMut/examples/../src/pyMut/data/resources
🗄️ Ruta de base de datos: /home/luisruimore/Escritorio/pyMut/examples/../src/pyMut/data/resources/data.duckdb


## 1. Construcción de Base de Datos Embebida

Primero construimos la base de datos DuckDB con:
- Datos de dominios PFAM (`Pfam-A.regions.tsv.gz`)
- Mappings UniProt (`HUMAN_9606_idmapping_selected.tab.gz`)
- Índices optimizados para consultas rápidas


In [3]:
# Construir base de datos (puede tomar varios minutos la primera vez)
print("🔨 Construyendo base de datos embebida...")
db_path = build_embedded_db(force_rebuild=False)  # Cambiar a True para forzar reconstrucción
print(f"✅ Base de datos creada en: {db_path}")


🔨 Construyendo base de datos embebida...
✅ Database already exists at /home/luisruimore/Escritorio/pyMut/examples/../src/pyMut/data/resources/data.duckdb
✅ Base de datos creada en: /home/luisruimore/Escritorio/pyMut/examples/../src/pyMut/data/resources/data.duckdb


In [4]:
# Conectar a la base de datos y verificar contenido
conn = connect_db()

print("📊 Estadísticas de la base de datos:")
print("\n🧬 Tabla PFAM (dominios):")
pfam_count = conn.execute("SELECT COUNT(*) FROM pfam").fetchone()[0]
print(f"  Total de anotaciones PFAM: {pfam_count:,}")

print("\n🔗 Tabla XREF (mappings):")
xref_count = conn.execute("SELECT COUNT(*) FROM xref").fetchone()[0]
ensp_count = conn.execute("SELECT COUNT(*) FROM xref WHERE prot_id LIKE 'ENSP%'").fetchone()[0]
np_count = conn.execute("SELECT COUNT(*) FROM xref WHERE prot_id LIKE 'NP_%'").fetchone()[0]
print(f"  Total de mappings: {xref_count:,}")
print(f"  ENSP (Ensembl): {ensp_count:,}")
print(f"  NP_ (RefSeq): {np_count:,}")

print("\n📋 Ejemplos de dominios PFAM:")
pfam_samples = conn.execute("SELECT * FROM pfam LIMIT 5").fetchall()
for row in pfam_samples:
    print(f"  {row[0]} [{row[1]}-{row[2]}] -> {row[3]} ({row[4]})")

print("\n📋 Ejemplos de mappings:")
xref_samples = conn.execute("SELECT * FROM xref LIMIT 5").fetchall()
for row in xref_samples:
    print(f"  {row[0]} -> {row[1]}")


📊 Estadísticas de la base de datos:

🧬 Tabla PFAM (dominios):
  Total de anotaciones PFAM: 114,918,760

🔗 Tabla XREF (mappings):
  Total de mappings: 191,661
  ENSP (Ensembl): 124,204
  NP_ (RefSeq): 67,457

📋 Ejemplos de dominios PFAM:
  A0A437CBF4 [9-69] -> PF02251 (PF02251)
  A0A8C8IVL5 [11-61] -> PF02251 (PF02251)
  A0A369K9H9 [3-50] -> PF02251 (PF02251)
  A0A8D0AWP4 [5-58] -> PF02251 (PF02251)
  A0A7M7G933 [3-63] -> PF02251 (PF02251)

📋 Ejemplos de mappings:
  NP_003395.1 -> P31946
  NP_647539.1 -> P31946
  NP_006752.1 -> P62258
  NP_003396.1 -> Q04917
  NP_036611.2 -> P61981


## 2. Ejemplo con archivo MAF (tcga_laml.maf.gz)

Los archivos MAF ya contienen información de proteínas, por lo que podemos saltar directamente a la anotación PFAM.


In [5]:
# Cargar archivo MAF de ejemplo
maf_file = Path('../src/pyMut/data/examples/tcga_laml.maf.gz')
print(f"📂 Cargando archivo MAF: {maf_file}")

# Leer MAF (tomar una muestra para el demo)
maf_df = pd.read_csv(maf_file, sep='\t', low_memory=False).head(1000)
print(f"📊 Cargadas {len(maf_df):,} variantes del MAF")
print(f"📋 Columnas disponibles: {list(maf_df.columns)}")

# Mostrar estructura del MAF
print("\n🔍 Primeras filas del MAF:")
display(maf_df[['Hugo_Symbol', 'Variant_Classification', 'Protein_Change']].head())


📂 Cargando archivo MAF: ../src/pyMut/data/examples/tcga_laml.maf.gz
📊 Cargadas 1,000 variantes del MAF
📋 Columnas disponibles: ['Hugo_Symbol', 'Entrez_Gene_Id', 'Center', 'NCBI_Build', 'Chromosome', 'Start_Position', 'End_position', 'Strand', 'Variant_Classification', 'Variant_Type', 'Reference_Allele', 'Tumor_Seq_Allele1', 'Tumor_Seq_Allele2', 'Tumor_Sample_Barcode', 'Protein_Change', 'i_TumorVAF_WU', 'i_transcript_name']

🔍 Primeras filas del MAF:


,Hugo_Symbol,Variant_Classification,Protein_Change
0,ABCA10,Splice_Site,p.K960R
1,ABCA4,Missense_Mutation,p.R1517H
2,ABCB11,Missense_Mutation,p.A1283V
3,ABCC11,Silent,p.I490I
4,ABCC3,Missense_Mutation,p.P1271S


In [6]:
# Preparar datos MAF para anotación PFAM
# El MAF ya tiene información de proteínas, necesitamos extraer la posición del aminoácido

def extract_aa_position(protein_change):
    """Extraer posición del aminoácido de Protein_Change o HGVSp."""
    if pd.isna(protein_change) or protein_change == '':
        return None

    # Buscar patrones como p.A123V, p.R456*, etc.
    import re
    match = re.search(r'p\.[A-Z]*(\d+)', str(protein_change))
    if match:
        return int(match.group(1))
    return None

# Preparar DataFrame para anotación PFAM
maf_for_pfam = maf_df.copy()

# Extraer posición del aminoácido de Protein_Change
maf_for_pfam['aa_pos'] = maf_for_pfam['Protein_Change'].apply(extract_aa_position)

# Usar Hugo_Symbol como protein_id para el mapping automático
maf_for_pfam['protein_id'] = maf_for_pfam['Hugo_Symbol']

# Filtrar filas con información válida
maf_valid = maf_for_pfam.dropna(subset=['aa_pos', 'protein_id'])
maf_valid = maf_valid[maf_valid['protein_id'] != '']

print(f"📊 Variantes válidas para anotación PFAM: {len(maf_valid):,}")
print("\n🔍 Ejemplos de datos preparados:")
display(maf_valid[['Hugo_Symbol', 'Protein_Change', 'aa_pos', 'protein_id']].head())


📊 Variantes válidas para anotación PFAM: 955

🔍 Ejemplos de datos preparados:


,Hugo_Symbol,Protein_Change,aa_pos,protein_id
0,ABCA10,p.K960R,960.0,ABCA10
1,ABCA4,p.R1517H,1517.0,ABCA4
2,ABCB11,p.A1283V,1283.0,ABCB11
3,ABCC11,p.I490I,490.0,ABCC11
4,ABCC3,p.P1271S,1271.0,ABCC3


In [7]:
# Anotar con dominios PFAM
print("🧬 Anotando variantes MAF con dominios PFAM...")
maf_annotated = annotate_pfam(maf_valid, conn, aa_column='aa_pos')

print(f"📊 Variantes anotadas: {len(maf_annotated):,}")
pfam_hits = maf_annotated.dropna(subset=['pfam_id'])
print(f"🎯 Variantes con dominios PFAM: {len(pfam_hits):,}")

if len(pfam_hits) > 0:
    print("\n🔍 Ejemplos de anotaciones PFAM:")
    display(pfam_hits[['Hugo_Symbol', 'Protein_Change', 'aa_pos', 'uniprot', 'pfam_id', 'pfam_name']].head(10))
else:
    print("⚠️ No se encontraron dominios PFAM para las variantes del MAF")


🧬 Anotando variantes MAF con dominios PFAM...
🔗 Mapping protein IDs to UniProt...
⚠️  No UniProt mappings found in xref table
   For demonstration, using a sample of known gene-to-UniProt mappings...
   Mapped 158/955 variants using sample mappings
🧬 Annotating 158 variants with Pfam domains...
✅ Found Pfam domains for 851/955 variants
📊 Variantes anotadas: 1,727
🎯 Variantes con dominios PFAM: 851

🔍 Ejemplos de anotaciones PFAM:


,Hugo_Symbol,Protein_Change,aa_pos,uniprot,pfam_id,pfam_name
302,CEBPA,p.317_318insM,317.0,P49715,PF07716,PF07716
304,CEBPA,p.310_311TQ>TKQNPQ,310.0,P49715,PF07716,PF07716
305,CEBPA,p.310_311TQ>TKQNPQ,310.0,P49715,PF07716,PF07716
308,CEBPA,p.T318del,318.0,P49715,PF07716,PF07716
309,CEBPA,p.R300C,300.0,P49715,PF07716,PF07716
311,CEBPA,p.R286fs,286.0,P49715,PF07716,PF07716
312,CEBPA,p.308_309insV,308.0,P49715,PF07716,PF07716
315,CEBPA,p.305_306insQ,305.0,P49715,PF07716,PF07716
317,CEBPA,p.307_309NVE>K,307.0,P49715,PF07716,PF07716
319,CEBPA,p.310_311TQ>TWQ,310.0,P49715,PF07716,PF07716


In [8]:
# Generar resumen de dominios PFAM (estilo maftools)
if len(pfam_hits) > 0:
    print("📈 Generando resumen de dominios PFAM...")

    # Resumen por dominio PFAM
    pfam_summary = pfam_domains(
        pfam_hits,
        aa_column='aa_pos',
        summarize_by='PfamDomain',
        top_n=10,
        include_synonymous=False,
        plot=False
    )

    print("\n🏆 Top 10 dominios PFAM más afectados:")
    display(pfam_summary)

    # Resumen por posición de aminoácido
    aa_summary = pfam_domains(
        pfam_hits,
        aa_column='aa_pos',
        summarize_by='AAPos',
        top_n=5,
        include_synonymous=False,
        plot=False
    )

    print("\n🎯 Top 5 posiciones de aminoácidos más afectadas:")
    display(aa_summary)
else:
    print("⚠️ No hay suficientes datos para generar resumen")


📈 Generando resumen de dominios PFAM...
📊 Top 6 PfamDomain annotations:
           Domain  Count  Percentage
PF00180 (PF00180)    622       73.09
PF07714 (PF07714)    198       23.27
PF00145 (PF00145)     12        1.41
PF07716 (PF07716)     12        1.41
PF17980 (PF17980)      5        0.59
PF00855 (PF00855)      2        0.24

🏆 Top 10 dominios PFAM más afectados:


,Domain,Count,Percentage
1,PF00180 (PF00180),622,73.09
3,PF07714 (PF07714),198,23.27
0,PF00145 (PF00145),12,1.41
4,PF07716 (PF07716),12,1.41
5,PF17980 (PF17980),5,0.59
2,PF00855 (PF00855),2,0.24


📊 Top 5 AAPos annotations:
    Position  Count  Percentage
O75874:132.0    324       38.07
P48735:140.0    289       33.96
P36888:835.0    196       23.03
P48735:172.0      9        1.06
Q9Y6K1:736.0      4        0.47

🎯 Top 5 posiciones de aminoácidos más afectadas:


,Position,Count,Percentage
0,O75874:132.0,324,38.07
4,P48735:140.0,289,33.96
2,P36888:835.0,196,23.03
5,P48735:172.0,9,1.06
26,Q9Y6K1:736.0,4,0.47


## 3. Ejemplo con archivo VCF (subset_50k_variants.vcf)

Los archivos VCF requieren anotación de efectos de variantes antes de la anotación PFAM.


In [9]:
# Cargar archivo VCF de ejemplo
vcf_file = Path('../src/pyMut/data/subsets/subset_50k_variants.vcf')
print(f"📂 Cargando archivo VCF: {vcf_file}")

# Leer VCF (tomar una muestra pequeña para el demo)
vcf_lines = []
with open(vcf_file, 'r') as f:
    for line in f:
        if not line.startswith('#'):
            vcf_lines.append(line.strip().split('\t'))
        if len(vcf_lines) >= 100:  # Solo 100 variantes para el demo
            break

# Crear DataFrame del VCF
vcf_columns = ['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT'] + [f'SAMPLE_{i}' for i in range(len(vcf_lines[0])-9)]
vcf_df = pd.DataFrame(vcf_lines, columns=vcf_columns[:len(vcf_lines[0])])

# Preparar datos básicos
vcf_df['POS'] = vcf_df['POS'].astype(int)
vcf_df = vcf_df[['CHROM', 'POS', 'REF', 'ALT']].copy()

print(f"📊 Cargadas {len(vcf_df):,} variantes del VCF")
print("\n🔍 Primeras filas del VCF:")
display(vcf_df.head())


📂 Cargando archivo VCF: ../src/pyMut/data/subsets/subset_50k_variants.vcf
📊 Cargadas 100 variantes del VCF

🔍 Primeras filas del VCF:


,CHROM,POS,REF,ALT
0,10,11501,C,A
1,10,36097,G,A
2,10,45900,C,T
3,10,47049,GGA,G
4,10,47064,ACCT,A


In [10]:
# Anotar efectos de variantes usando VariantAnnotator
print("🔬 Anotando efectos de variantes con VariantAnnotator...")
print("⚠️ Nota: Usando backend 'pyvep' (REST API). Para mejor rendimiento usar 'vep_cli' con cache local.")

# Crear anotador (usar pyvep para simplicidad, aunque es más lento)
annotator = VariantAnnotator(backend='pyvep')

# Tomar solo unas pocas variantes para el demo (pyvep es lento)
vcf_sample = vcf_df.head(10).copy()
print(f"📊 Anotando {len(vcf_sample)} variantes de muestra...")

try:
    vcf_annotated = annotator.annotate(vcf_sample)
    print(f"✅ Anotación completada: {len(vcf_annotated)} variantes")

    # Mostrar resultados de anotación
    if len(vcf_annotated) > 0:
        print("\n🔍 Columnas después de anotación:")
        print(list(vcf_annotated.columns))

        # Filtrar variantes con información de proteína
        protein_variants = vcf_annotated.dropna(subset=['uniprot', 'aa_pos'])
        print(f"\n🧬 Variantes con información de proteína: {len(protein_variants)}")

        if len(protein_variants) > 0:
            display(protein_variants[['CHROM', 'POS', 'REF', 'ALT', 'uniprot', 'aa_pos', 'hgvsp']].head())
        else:
            print("⚠️ No se encontraron variantes con información de proteína")
            print("💡 Esto puede ocurrir si las variantes no afectan regiones codificantes")
    else:
        print("❌ No se obtuvieron resultados de anotación")

except Exception as e:
    print(f"❌ Error durante anotación: {e}")
    print("💡 Esto puede ocurrir por problemas de conectividad o límites de API")
    print("💡 Para uso en producción, se recomienda usar backend 'vep_cli' con cache local")

    # Crear datos de ejemplo para continuar la demostración
    print("\n🔧 Creando datos de ejemplo para continuar la demostración...")
    vcf_annotated = vcf_sample.copy()
    vcf_annotated['uniprot'] = ['P04637', 'P53350', None, None, 'Q96QK1', None, None, None, 'P04637', None]
    vcf_annotated['aa_pos'] = [273, 175, None, None, 456, None, None, None, 248, None]
    vcf_annotated['hgvsp'] = ['p.R273H', 'p.R175H', None, None, 'p.G456A', None, None, None, 'p.R248Q', None]

    protein_variants = vcf_annotated.dropna(subset=['uniprot', 'aa_pos'])
    print(f"🧬 Variantes de ejemplo con información de proteína: {len(protein_variants)}")


🔬 Anotando efectos de variantes con VariantAnnotator...
⚠️ Nota: Usando backend 'pyvep' (REST API). Para mejor rendimiento usar 'vep_cli' con cache local.
📊 Anotando 10 variantes de muestra...
🌐 Annotating variants using Ensembl REST API...
⚠️  Note: REST API has rate limits. For large datasets, consider using 'vep_cli' backend.
  Processing batch 1/1 (10 variants)...
❌ Error durante anotación: 'chrom'
💡 Esto puede ocurrir por problemas de conectividad o límites de API
💡 Para uso en producción, se recomienda usar backend 'vep_cli' con cache local

🔧 Creando datos de ejemplo para continuar la demostración...
🧬 Variantes de ejemplo con información de proteína: 4


In [11]:
# Anotar variantes VCF con dominios PFAM
if 'protein_variants' in locals() and len(protein_variants) > 0:
    print("🧬 Anotando variantes VCF con dominios PFAM...")

    vcf_pfam = annotate_pfam(protein_variants, conn, aa_column='aa_pos')

    print(f"📊 Variantes anotadas: {len(vcf_pfam):,}")
    pfam_hits_vcf = vcf_pfam.dropna(subset=['pfam_id'])
    print(f"🎯 Variantes con dominios PFAM: {len(pfam_hits_vcf):,}")

    if len(pfam_hits_vcf) > 0:
        print("\n🔍 Ejemplos de anotaciones PFAM en VCF:")
        display(pfam_hits_vcf[['CHROM', 'POS', 'REF', 'ALT', 'uniprot', 'aa_pos', 'pfam_id', 'pfam_name']])

        # Generar resumen
        print("\n📈 Resumen de dominios PFAM en VCF:")
        vcf_pfam_summary = pfam_domains(
            pfam_hits_vcf,
            aa_column='aa_pos',
            summarize_by='PfamDomain',
            top_n=5,
            include_synonymous=False,
            plot=False
        )
        display(vcf_pfam_summary)
    else:
        print("⚠️ No se encontraron dominios PFAM para las variantes del VCF")
else:
    print("⚠️ No hay variantes con información de proteína para anotar")


🧬 Anotando variantes VCF con dominios PFAM...
🧬 Annotating 4 variants with Pfam domains...
✅ Found Pfam domains for 4/4 variants
📊 Variantes anotadas: 4
🎯 Variantes con dominios PFAM: 4

🔍 Ejemplos de anotaciones PFAM en VCF:


,CHROM,POS,REF,ALT,uniprot,aa_pos,pfam_id,pfam_name
0,10,11501,C,A,P04637,273.0,PF00870,PF00870
1,10,36097,G,A,P53350,175.0,PF00069,PF00069
2,10,47064,ACCT,A,Q96QK1,456.0,PF03635,PF03635
3,10,47317,G,A,P04637,248.0,PF00870,PF00870



📈 Resumen de dominios PFAM en VCF:
📊 Top 3 PfamDomain annotations:
           Domain  Count  Percentage
PF00870 (PF00870)      2        50.0
PF00069 (PF00069)      1        25.0
PF03635 (PF03635)      1        25.0


,Domain,Count,Percentage
1,PF00870 (PF00870),2,50.0
0,PF00069 (PF00069),1,25.0
2,PF03635 (PF03635),1,25.0


## 4. Comparación de Backends de Anotación

Demostración de las diferencias entre los backends `pyvep` y `vep_cli`.


In [12]:
# Comparar backends de anotación
print("⚖️ Comparación de backends de anotación:")
print("\n🐍 Backend 'pyvep':")
print("  ✅ Ventajas: Fácil de usar, no requiere instalación adicional")
print("  ❌ Desventajas: Lento (REST API), limitado por cuotas, requiere internet")
print("  🎯 Uso recomendado: Análisis exploratorio, pocas variantes")

print("\n⚡ Backend 'vep_cli':")
print("  ✅ Ventajas: Muy rápido, completo, funciona offline")
print("  ❌ Desventajas: Requiere instalación de VEP y cache")
print("  🎯 Uso recomendado: Análisis de producción, muchas variantes")

# Verificar disponibilidad de VEP CLI
try:
    vep_annotator = VariantAnnotator(backend='vep_cli')
    print("\n✅ VEP CLI está disponible y configurado")
except Exception as e:
    print(f"\n⚠️ VEP CLI no está disponible: {e}")
    print("💡 Para instalar VEP CLI, consultar: https://ensembl.org/info/docs/tools/vep/script/vep_download.html")


⚖️ Comparación de backends de anotación:

🐍 Backend 'pyvep':
  ✅ Ventajas: Fácil de usar, no requiere instalación adicional
  ❌ Desventajas: Lento (REST API), limitado por cuotas, requiere internet
  🎯 Uso recomendado: Análisis exploratorio, pocas variantes

⚡ Backend 'vep_cli':
  ✅ Ventajas: Muy rápido, completo, funciona offline
  ❌ Desventajas: Requiere instalación de VEP y cache
  🎯 Uso recomendado: Análisis de producción, muchas variantes
📦 Extracting VEP cache from /home/luisruimore/Escritorio/pyMut/examples/../src/pyMut/data/resources/vep/homo_sapiens_vep_113_GRCh38.tar.gz...
✅ VEP cache extracted successfully to /home/luisruimore/Escritorio/pyMut/examples/../src/pyMut/data/resources/vep
⚠️  VEP setup failed: VEP not found. Please install Ensembl VEP.
⚠️  Falling back to REST API backend (pyvep)

✅ VEP CLI está disponible y configurado


## 5. Análisis Avanzado de Dominios PFAM

Ejemplos de análisis más detallados usando las funciones de resumen.


In [13]:
# Análisis avanzado si tenemos datos anotados
if 'pfam_hits' in locals() and len(pfam_hits) > 0:
    print("📊 Análisis avanzado de dominios PFAM:")

    # Estadísticas generales
    total_variants = len(pfam_hits)
    unique_proteins = pfam_hits['uniprot'].nunique()
    unique_domains = pfam_hits['pfam_id'].nunique()

    print(f"\n📈 Estadísticas generales:")
    print(f"  Total de variantes con PFAM: {total_variants:,}")
    print(f"  Proteínas únicas afectadas: {unique_proteins:,}")
    print(f"  Dominios PFAM únicos: {unique_domains:,}")

    # Análisis por tipo de variante (si disponible)
    if 'Variant_Classification' in pfam_hits.columns:
        print(f"\n🔬 Distribución por tipo de variante:")
        variant_dist = pfam_hits['Variant_Classification'].value_counts().head()
        for variant_type, count in variant_dist.items():
            print(f"  {variant_type}: {count:,}")

    # Top proteínas más afectadas
    print(f"\n🧬 Top 5 proteínas más afectadas:")
    protein_counts = pfam_hits.groupby(['uniprot', 'Hugo_Symbol']).size().sort_values(ascending=False).head()
    for (uniprot, gene), count in protein_counts.items():
        print(f"  {gene} ({uniprot}): {count:,} variantes")

    # Análisis de cobertura de dominios
    print(f"\n🎯 Análisis de cobertura de dominios:")
    domain_coverage = pfam_hits.groupby('pfam_id').agg({
        'uniprot': 'nunique',
        'aa_pos': 'count',
        'pfam_name': 'first'
    }).sort_values('aa_pos', ascending=False).head()

    domain_coverage.columns = ['Proteínas', 'Variantes', 'Nombre']
    display(domain_coverage)

else:
    print("⚠️ No hay datos de PFAM suficientes para análisis avanzado")
    print("💡 Esto puede ocurrir si:")
    print("  - Las variantes no afectan regiones con dominios PFAM conocidos")
    print("  - Los mappings UniProt no están disponibles")
    print("  - La base de datos PFAM no está completa")


📊 Análisis avanzado de dominios PFAM:

📈 Estadísticas generales:
  Total de variantes con PFAM: 851
  Proteínas únicas afectadas: 5
  Dominios PFAM únicos: 6

🔬 Distribución por tipo de variante:
  Missense_Mutation: 831
  In_Frame_Ins: 8
  Nonsense_Mutation: 4
  Splice_Site: 3
  In_Frame_Del: 2

🧬 Top 5 proteínas más afectadas:
  IDH1 (O75874): 324 variantes
  IDH2 (P48735): 298 variantes
  FLT3 (P36888): 198 variantes
  DNMT3A (Q9Y6K1): 19 variantes
  CEBPA (P49715): 12 variantes

🎯 Análisis de cobertura de dominios:


,Proteínas,Variantes,Nombre
pfam_id,,,
PF00180,2,622,PF00180
PF07714,1,198,PF07714
PF00145,1,12,PF00145
PF07716,1,12,PF07716
PF17980,1,5,PF17980


## 6. Verificación de Recursos y Troubleshooting

Herramientas para verificar que todos los recursos están correctamente configurados.


In [14]:
# Verificación de recursos
print("🔍 Verificación de recursos:")

resources_path = get_resources_path()
print(f"\n📁 Directorio de recursos: {resources_path}")
print(f"   Existe: {'✅' if resources_path.exists() else '❌'}")

# Verificar archivos de recursos
resource_files = {
    'PFAM domains': resources_path / 'pfam' / 'Pfam-A.regions.tsv.gz',
    'UniProt mappings': resources_path / 'mappings' / 'HUMAN_9606_idmapping_selected.tab.gz',
    'Database': get_db_path()
}

print("\n📋 Archivos de recursos:")
for name, path in resource_files.items():
    exists = path.exists()
    size = f" ({path.stat().st_size / 1024 / 1024:.1f} MB)" if exists else ""
    print(f"  {name}: {'✅' if exists else '❌'}{size}")
    if not exists:
        print(f"    Ruta esperada: {path}")

# Verificar tablas de base de datos
if get_db_path().exists():
    print("\n🗄️ Tablas de base de datos:")
    try:
        tables = conn.execute("SHOW TABLES").fetchall()
        for table in tables:
            table_name = table[0]
            count = conn.execute(f"SELECT COUNT(*) FROM {table_name}").fetchone()[0]
            print(f"  {table_name}: {count:,} filas")
    except Exception as e:
        print(f"  ❌ Error accediendo a tablas: {e}")
else:
    print("\n❌ Base de datos no encontrada")

# Verificar archivos de ejemplo
example_files = {
    'MAF example': Path('../src/pyMut/data/examples/tcga_laml.maf.gz'),
    'VCF example': Path('../src/pyMut/data/subsets/subset_50k_variants.vcf')
}

print("\n📋 Archivos de ejemplo:")
for name, path in example_files.items():
    exists = path.exists()
    size = f" ({path.stat().st_size / 1024 / 1024:.1f} MB)" if exists else ""
    print(f"  {name}: {'✅' if exists else '❌'}{size}")


🔍 Verificación de recursos:

📁 Directorio de recursos: /home/luisruimore/Escritorio/pyMut/examples/../src/pyMut/data/resources
   Existe: ✅

📋 Archivos de recursos:
  PFAM domains: ✅ (4155.7 MB)
  UniProt mappings: ✅ (118.9 MB)
  Database: ✅ (7384.8 MB)

🗄️ Tablas de base de datos:
  meta: 2 filas
  pfam: 114,918,760 filas
  xref: 191,661 filas

📋 Archivos de ejemplo:
  MAF example: ✅ (0.1 MB)
  VCF example: ✅ (491.6 MB)


In [15]:
# Cerrar conexión a base de datos
if 'conn' in locals():
    conn.close()
    print("✅ Conexión a base de datos cerrada")


✅ Conexión a base de datos cerrada
